<a href="https://colab.research.google.com/github/WilliamZhang20/NLP_PyTorch/blob/master/translation_fr_en/TransformerEng_Fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I build a transformer using PyTorch to translate sentences from French to English, given a large text file of various translations.

In [6]:
!pip install torch torchvision torchaudio

In [10]:
from io import open
import unicodedata
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from collections import Counter

import numpy as np
import re

Now the transformer contains an encoder and a decoder. Unlike vanilla encoder/decoders with Recurrent Neural Networks (RNNs), the transformer both working in parallel.

However, to start off, we need the building blocks, the principal of which is multi-head attention.

This consists of multiple attention heads, as the name goes. A single attention head uses a set containing a query, key and value, which were learned during training.

A query is dataset dependent, and is part of the model's search for a pattern that commonly recurrs within sentences. Once the pattern is discovered, it applies attention weights to those parts of the sentence.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)